In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.draw import draw_squares
from utils.square import SquareDataset
from torch.utils.data import DataLoader
import utils.viz as torchviz
torchviz.init()

In [9]:
class LinearModel(nn.Module):
    def __init__(self, x, y):
        super(LinearModel, self).__init__()
        self.layer1 = nn.Linear(x, y)

    def forward(self, x):
        x = self.layer1(x)
        return x

In [10]:
squares = SquareDataset(60000)
for i in range(5):
    print(squares[i])

# Using the not-so-smart Model

In [11]:
model = LinearModel(9, 3)
X = squares[4][0].reshape(-1, 9)/255
print(X)
print(model(X))

In [12]:
model.layer1.weight, model.layer1.bias

# Looking at the gradients!

In [15]:
cost = torch.nn.MSELoss()
Y = squares[4][1].reshape(-1, 3)
loss = cost(model(X), Y)
print(loss)

In [16]:
torchviz.draw(loss)

# Optimizing All Teh Things!

In [17]:
# Use the nn package to define our model and loss function.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LinearModel(9, 3)
model = model.to(device)

cost = torch.nn.MSELoss()

# optimizer which Tensors it should update.
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# dataset!
dataloader = DataLoader(squares, batch_size=128)

epochs = 20

# The Optimization Loop

In [18]:
for t in range(epochs):
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device) / 255, Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

    print('l: {:>8f}, (e {:>3})'.format(loss.item(), t))

In [19]:
print("W's and b's:")
for p in model.parameters():
    print(p)

# Trying it out (inference)